# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [1]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [2]:
!pip install torch

In [2]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2662258
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [4]:
import os
import pandas as pd

os.mkdir('./outputs6')

Now, let's load some more packages we will need later:

In [5]:
!pip install datasets

  Using cached pyarrow-20.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-20.0.0-cp312-cp312-win_amd64.whl (25.7 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.1
    Uninstalling pyarrow-14.0.1:
      Successfully uninstalled pyarrow-14.0.1


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.


In [ ]:
pip install --upgrade pyarrow


In [7]:
!pip install pyarrow

In [8]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [9]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In [10]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\radha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

C:\Users\radha\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [12]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE    
    total_docs = len(labels)    #  for Total number of docs
    
    # For each class (e.g., 0, 1, 2), compute the prior probability
    for c in range(num_classes):
         count_c = np.sum(np.array(labels) == c)  # Count how many docs have label == c
         priors[c] = count_c / total_docs    # Compute prior: P(y = c) = count of class c / total docs

    return priors
    
class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.3525  0.39875 0.24875]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [13]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE    
    # Convert labels to a numpy array for easier indexing
    labels = np.array(labels)
    
    # Initialise class-wise total counts of all features (start with +1 smoothing)
    class_totals = np.ones(num_classes) * num_features  # Each class gets +1 for each feature (Laplace smoothing)
    
    # Loop over each document
    for i in range(X.shape[0]):
        label = labels[i]            # Class label for this document
        x_i = X[i]                   # Feature counts for this document
        
        # Add feature counts for this document to the corresponding class
        likelihoods[:, label] += x_i
        class_totals[label] += np.sum(x_i)  # Update total feature count for this class

    # Now convert counts to probabilities: P(x_i = w | y = c)
    for c in range(num_classes):
        likelihoods[:, c] /= class_totals[c]  # Normalize each column by total word count for class c

    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [14]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-6.49593806e-03 -5.03982446e+00 -2.20121208e+01]
 [-6.99174052e-12 -2.57530361e+01 -2.84682192e+01]
 [-4.09493369e-03 -5.50005533e+00 -1.79786317e+01]
 [-1.22953400e+00 -3.50318821e-01 -5.77386442e+00]
 [-8.04624621e+00 -8.07106294e-04 -7.62831753e+00]
 [ 0.00000000e+00 -4.62081160e+01 -4.98159045e+01]
 [-7.02005904e+00 -1.02006619e-03 -8.98102892e+00]
 [-5.15869989e-02 -3.17652871e+00 -4.76197412e+00]
 [-9.09473559e-02 -6.63957423e+00 -2.45775884e+00]
 [-2.02299793e-01 -1.69745946e+00 -1.32469884e+01]
 [-1.40202350e+00 -1.18123352e+00 -8.05193149e-01]
 [-1.12186528e+01 -5.44766287e+00 -4.32913614e-03]
 [-4.89565286e+00 -7.50864495e-03 -1.34096405e+01]
 [-3.48935506e+01 -9.47646287e-01 -4.90455140e-01]
 [-8.57662277e+00 -1.64377962e-01 -1.88789434e+00]
 [ 0.00000000e+00 -3.82589520e+01 -3.45901256e+01]
 [-1.62493090e+01 -4.25050835e+00 -1.43596807e-02]
 [-1.37284707e+01 -1.09125710e-06 -2.18743641e+01]
 [ 0.00000000e+00 -5.82646445e+01 -8.63369198e+01]
 [-3.52896017e+01  0.00000000e+

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.78

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [16]:
top_features = []

### WRITE YOUR ANSWER HERE
# Get feature names (n-grams) from the CountVectorizer used earlier
feature_names = np.array(vectorizer.get_feature_names_out())

# Get the likelihoods for class 0 (column 0 of the likelihoods matrix)
class_0_likelihoods = likelihoods[:, 0]

# Get the indices of the top 5 highest likelihood values for class 0
top_indices = np.argsort(class_0_likelihoods)[-5:][::-1]  # sort descending

top_features = feature_names[top_indices].tolist()     # Use these indices to get the actual n-gram feature strings

### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['climate change', ', and', 'of the', 'in the', ', the']


In [17]:
# Get all feature names (n-grams) from CountVectorizer
feature_names = np.array(vectorizer.get_feature_names_out())

# Store top 5 features per class in a dictionary for easy access & visualisation
top_features_per_class = {}

# Loop through each class (0, 1, 2)
for c in range(num_classes):
    # Get likelihoods for class c
    class_likelihoods = likelihoods[:, c]
    
    # Get indices of top 5 features for this class (sorted in descending order)
    top_indices = np.argsort(class_likelihoods)[-5:][::-1]
    
    # Store the corresponding feature names
    top_features_per_class[f'class_{c}'] = feature_names[top_indices].tolist()

# Print results
for c in range(num_classes):
    print(f"🔹 Top 5 n-grams for Class {c}: {top_features_per_class[f'class_{c}']}")
pd.DataFrame(top_features_per_class).to_csv('./outputs/11c_top_feats_all_classes.csv', index=False)


🔹 Top 5 n-grams for Class 0: ['climate change', ', and', 'of the', 'in the', ', the']
🔹 Top 5 n-grams for Class 1: ['of the', ', and', 'in the', 'climate change', '. the']
🔹 Top 5 n-grams for Class 2: ['of the', 'in the', ', we', ', and', '. the']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk


In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\radha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE
### Improvements:
# 1) Using unigrams + bigrams (ngram_range=(1,2))
# 2) Removing English stop words (helps reduce noise)
# 3) Lowercasing is done by default in CountVectorizer

    vectorizer = CountVectorizer(
        ngram_range=(1,2),                    # use unigrams and bigrams
        tokenizer=word_tokenize,             # same tokenizer
        stop_words=stopwords.words('english')  # remove common stop words
    )

    # Fit on training data and transform both sets
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]
    X_val = vectorizer.transform(val_texts).toarray()
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)

    ### END OF ANSWER
    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

0.75


I explored three ways in which I can improve the accuracy by changing the preprocessing:

Stemming: reduced the model’s accuracy from 0.795 to 0.72.

Stopword removal resulted in an accuracy of 0.70.

Adding unigrams (1,2-grams): preserved the baseline accuracy at 0.795, but did not improve it.

## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [22]:
!pip install transformers

In [23]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL

from torch import nn, optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

# Updated hyperparameters
sequence_length = 64
batch_size = 16

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length,
        return_tensors="pt"
    )

# Tokenize datasets
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

# Updated neural network with dropout + mean pooling
class FFTextClassifier(nn.Module):
    def __init__(self, vocab_size, sequence_length, num_classes, embedding_size=64, hidden_size=64):
        super(FFTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(embedding_size, hidden_size)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, input_words):
        embeds = self.embedding(input_words)                   # (batch_size, seq_len, embedding_size)
        pooled = embeds.mean(dim=1)                            # Mean pooling across sequence
        dropped = self.dropout(pooled)                         # Apply dropout
        x = self.activation(self.fc1(dropped))                 # Hidden layer + ReLU
        output = self.fc2(x)                                   # Output layer
        return output

    def run_training(self, train_loader, val_loader):
        num_epochs = 5
        learning_rate = 1e-3

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        dev_losses = []

        for epoch in range(num_epochs):
            self.train()
            train_losses = []

            for batch in train_loader:
                optimizer.zero_grad()
                output = self(batch['input_ids'])
                loss = loss_fn(output, batch['label'])
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {np.mean(train_losses):.4f}")

            self.eval()
            val_losses = []
            for batch in val_loader:
                with torch.no_grad():
                    val_output = self(batch['input_ids'])
                    val_loss = loss_fn(val_output, batch['label'])
                    val_losses.append(val_loss.item())

            avg_val_loss = np.mean(val_losses)
            dev_losses.append(avg_val_loss)
            print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_val_loss:.4f}")

        return dev_losses

# Prediction function
def predict_nn(trained_model, data_loader):
    trained_model.eval()
    preds = []
    for batch in data_loader:
        with torch.no_grad():
            output = trained_model(batch['input_ids'])
            preds.extend(output.argmax(dim=1).tolist())
    return preds

#  Correct vocab size
vocab_size = len(tokenizer.get_vocab())
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, num_classes)
dev_losses = nn_classifier_model.run_training(train_loader, val_loader)

# Get predictions
predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch 1/5 - Training Loss: 1.0772
Epoch 1/5 - Validation Loss: 1.0406
Epoch 2/5 - Training Loss: 1.0418
Epoch 2/5 - Validation Loss: 1.0135
Epoch 3/5 - Training Loss: 0.9977
Epoch 3/5 - Validation Loss: 0.9723
Epoch 4/5 - Training Loss: 0.9263
Epoch 4/5 - Validation Loss: 0.9064
Epoch 5/5 - Training Loss: 0.8298
Epoch 5/5 - Validation Loss: 0.8360
0.635


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [24]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [25]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch

# Load pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# Select a document (you can change this if needed)
document_text = train_texts[0]

# Tokenize the document and get input tensors
inputs = tokenizer(document_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Forward pass through BERT (no gradients needed)
with torch.no_grad():
    outputs = model(**inputs)

token_embeddings = outputs.last_hidden_state   # Get the last hidden state for all tokens: shape = (1, seq_len, hidden_size)

attention_mask = inputs["attention_mask"]     # Compute the attention mask to ignore padding tokens

# Expand mask to match token embeddings shape, then use it to zero out padding tokens
mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
masked_embeddings = token_embeddings * mask_expanded

# Mean pooling: sum embeddings and divide by valid token count
sum_embeddings = masked_embeddings.sum(dim=1)
token_counts = attention_mask.sum(dim=1).unsqueeze(-1)
mean_embedding = sum_embeddings / token_counts

doc_emb = mean_embedding.squeeze().tolist()   # Convert to list for saving

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[0.19846706092357635, 1.6071665287017822, -0.9995077252388, -0.3544827103614807, 0.657346785068512, 0.14188675582408905, -0.15870743989944458, 0.8339238166809082, -1.0900856256484985, 0.2886713445186615, 1.4965959787368774, 0.5616402626037598, -0.40516898036003113, -0.6598829030990601, 0.9313555359840393, -0.4718971848487854, -1.4062392711639404, -0.15950019657611847, -1.1501203775405884, 0.7429781556129456, 0.21735608577728271, 0.261880487203598, 1.1443134546279907, 1.2718770503997803, 2.103933334350586, -0.47337818145751953, 0.8963171243667603, -0.499630331993103, 0.00459488108754158, 0.2827831506729126, -0.7183407545089722, -2.034816265106201, -0.21384984254837036, -0.8904367685317993, 0.5413885712623596, -0.7581331729888916, -1.0007671117782593, -0.07442563027143478, -1.8278430700302124, -0.18822462856769562, 0.792319118976593, 0.12840747833251953, -0.11756368726491928, -1.4401206970214844, 0.11083000898361206, -2.0369434356689453, -0.7863167524337769, 0.5593069195747375, -0.560726

## TASK 1.3b

Using the same document embeddings method as the previous task (1.3a), find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors.  (2 marks)

In [26]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# Helper function: returns mean-pooled BERT embedding
def get_doc_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]
    mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    masked_embeddings = token_embeddings * mask_expanded
    sum_embeddings = masked_embeddings.sum(dim=1)
    token_counts = attention_mask.sum(dim=1).unsqueeze(-1)
    mean_embedding = sum_embeddings / token_counts
    return mean_embedding.squeeze()

# Get embedding for the chosen document
chosen_embedding = get_doc_embedding(chosen_document)

# Loop through all validation texts to find the most similar one
max_similarity = -1
most_similar_doc = ""

for text in val_texts:
    emb = get_doc_embedding(text)
    similarity = F.cosine_similarity(chosen_embedding, emb, dim=0).item()
    if similarity > max_similarity:
        max_similarity = similarity
        most_similar_doc = text

most_similar_doc = ""  # to store the most similar document

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

There is also increased public focus, including by governmental and non-governmental organizations, on these and other environmental sustainability matters, including deforestation and land use. Our reputation could be damaged if we or others in our industry do not act, or are perceived not to act, responsibly with respect to our impact on the environment.



## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch import nn, optim
import torch
import numpy as np
import random
from sklearn.metrics import accuracy_score

#on every run it's getting different accuracy, so I tried this
seed = my_student_number
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

# Tokenize dataset
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_tokens = train_dataset.map(tokenize_fn, batched=True)
val_tokens = val_dataset.map(tokenize_fn, batched=True)
train_tokens.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_tokens.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Dataloaders
train_loader = DataLoader(train_tokens, batch_size=16, shuffle=True)
val_loader = DataLoader(val_tokens, batch_size=32)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training setup
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)  # halve lr every 2 epochs
epochs = 5

best_val_loss = float('inf')
best_model_state = None

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {total_loss / len(train_loader):.4f}")

    # Validation loss
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{epochs} - Validation Loss: {val_loss:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()

# Load best model
model.load_state_dict(best_model_state)

# Predict on validation set
model.eval()
predictions_bert = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().tolist()
        predictions_bert.extend(preds)

# Save predictions and calculate accuracy
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_bert = accuracy_score(val_tokens["label"], predictions_bert)
print(f"Accuracy: {accuracy_bert:.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch 1/5 - Training Loss: 0.8597
Epoch 1/5 - Validation Loss: 0.6189
Epoch 2/5 - Training Loss: 0.4981
Epoch 2/5 - Validation Loss: 0.4876
Epoch 3/5 - Training Loss: 0.3237
Epoch 3/5 - Validation Loss: 0.4497
Epoch 4/5 - Training Loss: 0.2373
Epoch 4/5 - Validation Loss: 0.4584
Epoch 5/5 - Training Loss: 0.1666
Epoch 5/5 - Validation Loss: 0.4698
Accuracy: 0.8200
